# Byg en regressionsmodel: forbered og visualiser data

## **Lineær regression for græskar - Lektion 2**
#### Introduktion

Nu hvor du har de nødvendige værktøjer til at begynde at bygge maskinlæringsmodeller med Tidymodels og Tidyverse, er du klar til at begynde at stille spørgsmål til dine data. Når du arbejder med data og anvender ML-løsninger, er det meget vigtigt at forstå, hvordan man stiller de rigtige spørgsmål for korrekt at udnytte potentialet i dit datasæt.

I denne lektion vil du lære:

-   Hvordan du forbereder dine data til modelbygning.

-   Hvordan du bruger `ggplot2` til datavisualisering.

Det spørgsmål, du ønsker besvaret, vil afgøre, hvilken type ML-algoritmer du skal anvende. Og kvaliteten af det svar, du får tilbage, vil i høj grad afhænge af karakteren af dine data.

Lad os se dette ved at arbejde igennem en praktisk øvelse.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>


<!--![Kunstværk af \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Kunstværk af \@allison_horst-->


## 1. Importering af græskardata og tilkaldelse af Tidyverse

Vi skal bruge følgende pakker for at skære og analysere denne lektion:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling af R-pakker](https://www.tidyverse.org/packages), der er designet til at gøre datavidenskab hurtigere, nemmere og sjovere!

Du kan installere dem med følgende kommando:

`install.packages(c("tidyverse"))`

Scriptet nedenfor tjekker, om du har de nødvendige pakker til at gennemføre dette modul, og installerer dem for dig, hvis nogle mangler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nu skal vi fyre op for nogle pakker og indlæse [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), der er leveret til denne lektion!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

En hurtig `glimpse()` viser straks, at der er tomme felter og en blanding af strenge (`chr`) og numeriske data (`dbl`). `Date` er af typen karakter, og der er også en mærkelig kolonne kaldet `Package`, hvor dataene er en blanding af `sacks`, `bins` og andre værdier. Dataene er faktisk lidt af en rodebunke 😤.

Det er faktisk ikke særlig almindeligt at få foræret et datasæt, der er helt klar til brug for at skabe en ML-model direkte. Men bare rolig, i denne lektion vil du lære, hvordan du forbereder et råt datasæt ved hjælp af standard R-biblioteker 🧑‍🔧. Du vil også lære forskellige teknikker til at visualisere data. 📈📊
<br>

> En genopfriskning: Pipe-operatoren (`%>%`) udfører operationer i logisk rækkefølge ved at sende et objekt videre ind i en funktion eller et kald. Du kan tænke på pipe-operatoren som at sige "og så" i din kode.


## 2. Kontroller for manglende data

En af de mest almindelige udfordringer, som dataforskere skal håndtere, er ufuldstændige eller manglende data. R repræsenterer manglende eller ukendte værdier med en speciel sentinelværdi: `NA` (Not Available).

Så hvordan kan vi finde ud af, om dataframen indeholder manglende værdier?
<br>
-   En direkte metode ville være at bruge base R-funktionen `anyNA`, som returnerer de logiske objekter `TRUE` eller `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Fantastisk, det ser ud til, at der mangler nogle data! Det er et godt sted at starte.

-   En anden måde ville være at bruge funktionen `is.na()`, som angiver, hvilke individuelle kolonneelementer der mangler, med en logisk `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okay, fik jobbet gjort, men med en stor dataramme som denne ville det være ineffektivt og praktisk talt umuligt at gennemgå alle rækker og kolonner individuelt😴.

-   En mere intuitiv måde ville være at beregne summen af de manglende værdier for hver kolonne:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Meget bedre! Der mangler data, men måske vil det ikke have betydning for opgaven. Lad os se, hvad yderligere analyse bringer frem.

> Sammen med de fantastiske sæt af pakker og funktioner har R en meget god dokumentation. For eksempel kan du bruge `help(colSums)` eller `?colSums` for at finde ud af mere om funktionen.


## 3. Dplyr: En grammatik for datamanipulation

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>


<!--![Kunstværk af \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Kunstværk af \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), en pakke i Tidyverse, er en grammatik for datamanipulation, der tilbyder et konsistent sæt af verber, som hjælper dig med at løse de mest almindelige udfordringer inden for datamanipulation. I denne sektion vil vi udforske nogle af dplyrs verber!  
<br>


#### dplyr::select()

`select()` er en funktion i pakken `dplyr`, som hjælper dig med at vælge kolonner, du vil beholde eller udelade.

For at gøre din data frame lettere at arbejde med, kan du fjerne flere af dens kolonner ved hjælp af `select()` og kun beholde de kolonner, du har brug for.

For eksempel vil vores analyse i denne øvelse involvere kolonnerne `Package`, `Low Price`, `High Price` og `Date`. Lad os vælge disse kolonner.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` er en funktion i pakken `dplyr`, som hjælper dig med at oprette eller ændre kolonner, samtidig med at de eksisterende kolonner bevares.

Den generelle struktur for `mutate` er:

`data %>%   mutate(new_column_name = what_it_contains)`

Lad os prøve `mutate` af ved at bruge `Date`-kolonnen og udføre følgende operationer:

1.  Konverter datoerne (som i øjeblikket er af typen karakter) til et månedsformat (disse er amerikanske datoer, så formatet er `MM/DD/YYYY`).

2.  Uddrag måneden fra datoerne til en ny kolonne.

I R gør pakken [lubridate](https://lubridate.tidyverse.org/) det nemmere at arbejde med dato-tidsdata. Så lad os bruge `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` og se, hvordan vi kan opnå ovenstående mål. Vi kan fjerne `Date`-kolonnen, da vi ikke får brug for den igen i de efterfølgende operationer.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Lad os nu oprette en ny kolonne `Price`, som repræsenterer gennemsnitsprisen på et græskar. For at udfylde den nye Price-kolonne tager vi gennemsnittet af kolonnerne `Low Price` og `High Price`.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Men vent lige!", vil du sige efter at have skimmet hele datasættet med `View(pumpkins)`, "Der er noget mærkeligt her!"🤔

Hvis du kigger på kolonnen `Package`, bliver græskar solgt i mange forskellige konfigurationer. Nogle sælges i mål af `1 1/9 bushel`, nogle i `1/2 bushel`, nogle per græskar, nogle per pund, og nogle i store kasser med varierende bredder.

Lad os verificere dette:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Fantastisk!👏

Græskar ser ud til at være meget svære at veje konsekvent, så lad os filtrere dem ved kun at vælge græskar med strengen *bushel* i `Package`-kolonnen og lægge dette i en ny data frame `new_pumpkins`.  
<br>


#### dplyr::filter() og stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): opretter et datasæt, der kun indeholder **rækker**, som opfylder dine betingelser, i dette tilfælde græskar med teksten *bushel* i kolonnen `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): registrerer tilstedeværelsen eller fraværet af et mønster i en tekst.

Pakken [`stringr`](https://github.com/tidyverse/stringr) tilbyder simple funktioner til almindelige tekstoperationer.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Du kan se, at vi har indsnævret det til omkring 415 rækker med data, der indeholder græskar i store mængder.🤩  
<br>


#### dplyr::case_when()

**Men vent! Der er én ting mere, du skal gøre**

Lagde du mærke til, at mængden af bushels varierer fra række til række? Du skal normalisere prissætningen, så du viser prisen per bushel og ikke per 1 1/9 eller 1/2 bushel. Det er tid til at lave lidt matematik for at standardisere det.

Vi bruger funktionen [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) til at *mutere* kolonnen Price afhængigt af nogle betingelser. `case_when` giver dig mulighed for at vektorisere flere `if_else()`-udsagn.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nu kan vi analysere prisen pr. enhed baseret på deres bushel-mål. Hele denne undersøgelse af bushels af græskar viser dog, hvor `vigtigt` det er at `forstå naturen af dine data`!

> ✅ Ifølge [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) afhænger en bushels vægt af typen af afgrøde, da det er en volumenmåling. "En bushel tomater, for eksempel, skal veje 56 pund... Blade og grønt fylder mere med mindre vægt, så en bushel spinat vejer kun 20 pund." Det er alt sammen ret kompliceret! Lad os undlade at lave en bushel-til-pund-konvertering og i stedet prissætte efter bushel. Hele denne undersøgelse af bushels af græskar viser dog, hvor vigtigt det er at forstå naturen af dine data!
>
> ✅ Lagde du mærke til, at græskar solgt pr. halv-bushel er meget dyre? Kan du finde ud af hvorfor? Hint: små græskar er langt dyrere end store, sandsynligvis fordi der er så mange flere af dem pr. bushel, givet den ubrugte plads, som ét stort, hult tærtegræskar optager.


Vigtigt, lad os også flytte kolonnen "Month" til den første position, altså **før** kolonnen "Package".

`dplyr::relocate()` bruges til at ændre kolonnepositioner.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Godt klaret!👌 Du har nu et rent og ryddeligt datasæt, som du kan bruge til at bygge din nye regressionsmodel!  
<br>


## 4. Datavisualisering med ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografik af Dasani Madipalli</figcaption>


<!--![Infografik af Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Der er et *klogt* ordsprog, der lyder sådan her:

> "Den simple graf har bragt mere information til dataanalytikerens sind end noget andet værktøj." --- John Tukey

En del af dataforskerens rolle er at demonstrere kvaliteten og karakteren af de data, de arbejder med. For at gøre dette skaber de ofte interessante visualiseringer, eller grafer, diagrammer og plots, der viser forskellige aspekter af dataene. På denne måde kan de visuelt vise relationer og mangler, som ellers kan være svære at opdage.

Visualiseringer kan også hjælpe med at afgøre, hvilken maskinlæringsteknik der er mest passende for dataene. Et spredningsdiagram, der ser ud til at følge en linje, indikerer for eksempel, at dataene er en god kandidat til en lineær regressionsøvelse.

R tilbyder flere systemer til at lave grafer, men [`ggplot2`](https://ggplot2.tidyverse.org/index.html) er en af de mest elegante og alsidige. `ggplot2` giver dig mulighed for at sammensætte grafer ved at **kombinere uafhængige komponenter**.

Lad os starte med et simpelt spredningsdiagram for kolonnerne Price og Month.

I dette tilfælde starter vi med [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), angiver et datasæt og æstetisk mapping (med [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) og tilføjer derefter lag (som [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) for spredningsdiagrammer.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Er dette et nyttigt plot 🤷? Er der noget ved det, der overrasker dig?

Det er ikke specielt nyttigt, da det blot viser dine data som en spredning af punkter i en given måned.  
<br>


### **Hvordan gør vi det nyttigt?**

For at få grafer til at vise nyttige data, skal du normalt gruppere dataene på en eller anden måde. For eksempel i vores tilfælde vil det at finde gennemsnitsprisen på græskar for hver måned give mere indsigt i de underliggende mønstre i vores data. Dette leder os til endnu en **dplyr** gennemgang:

#### `dplyr::group_by() %>% summarize()`

Grupperet aggregering i R kan nemt beregnes ved hjælp af

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` ændrer analyseenheden fra det komplette datasæt til individuelle grupper, såsom pr. måned.

-   `dplyr::summarize()` opretter en ny data frame med én kolonne for hver grupperingsvariabel og én kolonne for hver af de opsummeringsstatistikker, du har angivet.

For eksempel kan vi bruge `dplyr::group_by() %>% summarize()` til at gruppere græskarrene i grupper baseret på **Month**-kolonnen og derefter finde **gennemsnitsprisen** for hver måned.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Kort og godt!✨

Kategoriske funktioner som måneder repræsenteres bedst ved hjælp af et søjlediagram 📊. De lag, der bruges til søjlediagrammer, er `geom_bar()` og `geom_col()`. Se `?geom_bar` for at lære mere.

Lad os lave et!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Dette er en mere nyttig datavisualisering! Det ser ud til at indikere, at den højeste pris for græskar forekommer i september og oktober. Stemmer det overens med dine forventninger? Hvorfor eller hvorfor ikke?

Tillykke med at have afsluttet den anden lektion 👏! Du forberedte dine data til modelbygning og opdagede derefter flere indsigter ved hjælp af visualiseringer!



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
